### Step 1: Creating tokens

In [15]:
# Regular Expression Library
import re 

book_path = "the-verdict.txt"

with open(book_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Total number of characters: {len(raw_text)}")
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


#### Example of splitting text

In [16]:
example_text = "Hello, world. This, is a test."
example_result = re.split(r'(\s)', example_text) # Splits the text at spaces '\s' and includes the spaces.

print(example_result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [17]:
example_result = re.split(r'([,.]|\s)', example_text) # Including ',' and '.' in the split.

print(example_result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [18]:
example_result = [item for item in example_result if item.strip()] # Removes spaces

print(example_result)

### Removing white spaces or not depends on the type of application. 
### Removing white spaces reduces memory and computing requirements.
### Keeping whitespaces is useful if we train models which are sensitive to the structure of the text. E.g. Python code indentation.

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [19]:
example_text = "Hello, world. Is this-- a test?"
example_result = re.split(r'([,.:;?_!"()\']|--|\s)', example_text)
example_result = [item.strip() for item in example_result if item.strip()]
print(example_result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


#### Tokenizing the complete raw text

In [20]:
preprocessed_text = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed_text = [item.strip() for item in preprocessed_text if item.strip()]
print(f"Length of preprocessed text: {len(preprocessed_text)}")
print(preprocessed_text[:30])

Length of preprocessed text: 4690
['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


### Step 2: Creating token IDs

In [21]:
# Creating a list of unique tokens and sorting them alphabetically to determine vocabulary size.

unique_words = sorted(set(preprocessed_text))
vocabulary_size = len(unique_words)

print(f"The length of the vocabulary is: {vocabulary_size}")

The length of the vocabulary is: 1130


In [22]:
vocabulary = {token:integer for integer, token in enumerate(unique_words)}

vocabulary

{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 ',': 5,
 '--': 6,
 '.': 7,
 ':': 8,
 ';': 9,
 '?': 10,
 'A': 11,
 'Ah': 12,
 'Among': 13,
 'And': 14,
 'Are': 15,
 'Arrt': 16,
 'As': 17,
 'At': 18,
 'Be': 19,
 'Begin': 20,
 'Burlington': 21,
 'But': 22,
 'By': 23,
 'Carlo': 24,
 'Chicago': 25,
 'Claude': 26,
 'Come': 27,
 'Croft': 28,
 'Destroyed': 29,
 'Devonshire': 30,
 'Don': 31,
 'Dubarry': 32,
 'Emperors': 33,
 'Florence': 34,
 'For': 35,
 'Gallery': 36,
 'Gideon': 37,
 'Gisburn': 38,
 'Gisburns': 39,
 'Grafton': 40,
 'Greek': 41,
 'Grindle': 42,
 'Grindles': 43,
 'HAD': 44,
 'Had': 45,
 'Hang': 46,
 'Has': 47,
 'He': 48,
 'Her': 49,
 'Hermia': 50,
 'His': 51,
 'How': 52,
 'I': 53,
 'If': 54,
 'In': 55,
 'It': 56,
 'Jack': 57,
 'Jove': 58,
 'Just': 59,
 'Lord': 60,
 'Made': 61,
 'Miss': 62,
 'Money': 63,
 'Monte': 64,
 'Moon-dancers': 65,
 'Mr': 66,
 'Mrs': 67,
 'My': 68,
 'Never': 69,
 'No': 70,
 'Now': 71,
 'Nutley': 72,
 'Of': 73,
 'Oh': 74,
 'On': 75,
 'Once': 76,
 'Only': 77,
 '

In [23]:
# Creating a class for the tokenizer

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replacing spaces before the punctuations.
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text)
        return text

In [24]:
tokenizer = SimpleTokenizerV1(vocabulary)

text = """"It's the last he painted, you know,"
            Mrs. Gisburn said with pardonable pride."""

ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [25]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [26]:
## If there is a word that is not present in the dictionary (training dataset) an error is shown when encoding that text.
## In order to counter this error, we need to add special context tokens.

all_tokens = sorted(set(preprocessed_text))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocabulary = {token:integer for integer, token in enumerate(all_tokens)}

len(vocabulary.items())

1132

In [27]:
# Creating a class for the tokenizer which handles unknown words.

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replacing spaces before the punctuations.
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1', text)
        return text

In [28]:
tokenizer = SimpleTokenizerV2(vocabulary)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [29]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [30]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

### Byte pair Encoding

In [31]:
! pip install tiktoken

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/5b/64/b16003419a1d7728d0d8c0d56a4c24325e7b10a21a9dd1fc0f7115c02f0a/tiktoken-0.9.0-cp312-cp312-win_amd64.whl.metadata
  Using cached tiktoken-0.9.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/38/ec/ad2d7de49a600cdb8dd78434a1aeffe28b9d6fc42eb36afab4a27ad23384/regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata
  Using cached regex-2024.11.6-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Obtaining dependency information for requests>=2.26.0 from https://files.pythonhosted.org/packages/f9/9b/335f9764261e915ed497fcdeb11df5dfd6f7bf257d4a6a2a686d80da4d54/requests-2.32.3-py3-none-any.whl.metadata
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Obtaining dependency information for charset-normalizer<4,>=2 from https://files.pythonhosted.org/packages/6c/4f/c02d5c493967af3eda9c771ad4d2bbc8


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [32]:
import tiktoken

In [33]:
tokenizer = tiktoken.get_encoding("gpt2")

In [34]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
    "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [35]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [36]:
# Example handling random words.

integers = tokenizer.encode("Akdwu bjos")
print(integers)

strings = tokenizer.decode(integers)
print(strings)

[33901, 67, 43812, 275, 73, 418]
Akdwu bjos
